In [55]:
import openai
import panel as pn
import os
import base64
import json
import inspect
from openai import OpenAI
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from bs4 import BeautifulSoup
from functools import wraps

In [41]:
openai.__version__

'1.109.1'

In [42]:
with open ('/content/drive/MyDrive/APi_Keys/API_Key_Open_AI.txt') as f:
  api_key = f.readline().strip()

In [43]:
client = OpenAI(api_key=api_key)

In [44]:
Credential = '/content/drive/MyDrive/APi_Keys/client_secret_.apps.googleusercontent.com.json'
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

## Funciones

In [45]:
def _ejecutar_tool(tool_call):
    name = getattr(tool_call.function, "name", "")
    args_str = getattr(tool_call.function, "arguments", "") or "{}"
    try:
        args = json.loads(args_str)
    except Exception:
        args = {}
    if name == 'obtener_correos':
        return obtener_correos(remitente=args.get('remitente', '')) or ''
    return ""

In [56]:
spinner = pn.indicators.LoadingSpinner(value=False, width=30, height=30)  # ⏳
status  = pn.pane.Markdown("", sizing_mode="stretch_width")               # línea de estado opcional

_BUSY_COUNT = {"n": 0}

def _set_busy(active: bool, msg: str = ""):
    try:
        spinner.value = active
        status.object = msg if active else ""
    except Exception:
        pass  # evita que un error visual deje colgado el flujo

def with_busy(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        _BUSY_COUNT["n"] += 1
        _set_busy(True, "⏳ Procesando...")

        def _finish():
            _BUSY_COUNT["n"] = max(0, _BUSY_COUNT["n"] - 1)
            if _BUSY_COUNT["n"] == 0:
                _set_busy(False, "")

        try:
            result = fn(*args, **kwargs)

            # Si la función devolviera un awaitable (poco común en on_click),
            # apagamos el spinner cuando termine.
            if inspect.isawaitable(result):
                async def _await_and_finish():
                    try:
                        return await result
                    finally:
                        _finish()
                # Importante: devolver el awaitable para que Panel lo procese si corresponde
                return _await_and_finish()

            return result
        except Exception as e:
            # Muestra el error en tu columna de conversación si la tienes
            try:
                Interactive_conversation.append(
                    pn.Row('Error:', pn.pane.Markdown(str(e), width=600))
                )
            finally:
                _finish()
            # Relevanta el error si quieres que burbujee
            # raise
        finally:
            # Si no era awaitable, este finally apagará el spinner;
            # si era awaitable, _finish lo hará al terminar.
            if not inspect.isawaitable(locals().get("result", None)):
                _finish()
    return wrapper

In [47]:
def obtener_correos(remitente):
  creds = None
  if os.path.exists('/content/drive/MyDrive/APi_Keys/token.json'):
      creds = Credentials.from_authorized_user_file('/content/drive/MyDrive/APi_Keys/token.json', SCOPES)
  try:
      correos = """"""
      service = build('gmail', 'v1', credentials=creds)
      results = service.users().messages().list(userId='me', labelIds=['INBOX']).execute()
      for msg in results.get('messages',[]):
          message = service.users().messages().get(userId='me', id=msg['id']).execute()
          headers = message['payload'].get('headers',[])
          from_header = next((h['value'] for h in headers if h['name'] == 'From'), None)
          if from_header and remitente.strip().lower() in from_header.lower():
            parts = message['payload'].get('parts',[])
            for part in parts:
              if part['mimeType'] == 'text/plain':
                datos = part['body'].get('data')
                if datos:
                  decoded = base64.urlsafe_b64decode(datos).decode('utf-8')
                  correos += f"\n\n--- Correo ---\n{decoded}\n"

      if not correos:
          print('No se encontraron correos del remitente especificado')
      return correos
  except HttpError as error:
      print(f'Ocurrió un error de la API: {error}')


In [48]:
tools = [
     {
     "type":"function",
     "function":{
            "name": "obtener_correos",
            "description": "Obtiene los correos electrónicos de un remitente específico.",
            "parameters": {
                "type" : "object",
                "properties": {
                    "remitente": {
                        "type": "string",
                        "description": "El remitente del correo electrónico."
                        }
                    },
                "required": ["remitente"]
            },
        },
      }
]

In [49]:
def obtener_completion(mensajes, model = 'gpt-4o'):
  respuesta = client.chat.completions.create(
      model = model,
      messages = mensajes,
      tools = tools,
      tool_choice = 'auto',
      temperature = 0,
  )
  return respuesta.choices[0].message

In [50]:
def _ejecutar_tool(tool_call):
  name = tool_call.function.name
  args = json.loads(tool_call.function.arguments or "{}")
  if name == 'obtener_correos':
    return obtener_correos(remitente = args.get('remitente','')) or ''

In [51]:
@with_busy
def end_chat(event):
  Interactive_conversation.append(pn.pane.Alert("Chat terminado por el usuario", alert_type = 'success'))
  context.append({'role':'system','content': 'Despide al usuario de forma breve y amable. No hagas nada mas.'})
  response = obtener_completion(context)
  Interactive_conversation.append(
        pn.Row('Assistant:', pn.pane.Markdown(response.content, width=600,
                              styles={'background-color':'#F6F6F6','padding':'10px','border-radius':'5px'}))
    )

In [52]:
@with_busy
def collect_messages(event=None):
    prompt = inp.value
    inp.value = ''
    context.append({'role':'user', 'content': prompt})
    try:
        respuesta = obtener_completion(context)

        if getattr(respuesta, "tool_calls", None):
              context.append({
                "role": "assistant",
                "content": respuesta.content or "",
                "tool_calls": [
                    {
                        "id": tc.id,
                        "type": tc.type,  # "function"
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments
                        }
                    }
                    for tc in respuesta.tool_calls
                ]
            })

            # 3) Ejecutar cada tool_call y agregar su 'role: tool'
              for tc in respuesta.tool_calls:
                tool_output = _ejecutar_tool(tc) or ""
                context.append({
                    "role": "tool",
                    "tool_call_id": tc.id,
                    "name": tc.function.name,
                    "content": str(tool_output)  # siempre string
                })


          # 3) segundo turno con resultados de herramientas
              respuesta_2 = obtener_completion(context)
              context.append({"role": "assistant", "content": respuesta_2.content})
              Interactive_conversation.append(
                  pn.Row('Assistant:', pn.pane.Markdown(respuesta_2.content or "(sin contenido)", width=600,
                      styles={'background-color':'white','padding':'10px','border-radius':'5px'}))
              )
        else:
                  # respuesta directa
                context.append({"role": "assistant", "content": respuesta.content})
                Interactive_conversation.append(
                    pn.Row('Assistant:', pn.pane.Markdown(respuesta.content or "(sin contenido)", width=600,
                    styles={'background-color':'white','padding':'10px','border-radius':'5px'}))
                  )
    except Exception as e:
            Interactive_conversation.append(pn.Row('Error:', pn.pane.Markdown(str(e), width=600)))


In [57]:
pn.extension(notifications = True)
panels = []
context = [ {'role':'system', 'content':
"""
Eres un asistente virtual para gestionar y procesar el correo electrónico. \
Interactúa amablemente con el usuario y solicítale el correo electrónico de un \
remitente para obtener sus correos y empezar a trabajar sobre ellos.
"""
} ]

inp = pn.widgets.TextInput(placeholder = 'Escribe tu mensaje aquí...')
button_conversation = pn.widgets.Button(name = 'Enviar')
button_end_chat = pn.widgets.Button(name = 'Terminar Chat')
Interactive_conversation = pn.Column(sizing_mode="stretch_width")

button_conversation.on_click(collect_messages)
button_end_chat.on_click(end_chat)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation, button_end_chat, spinner),
    Interactive_conversation,  # <- montas el contenedor directo
    sizing_mode="stretch_both"
)

dashboard


/tmp/ipython-input-360001107.py:1: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension(notifications = True)


NotificationArea()

Column(sizing_mode='stretch_both')
    [0] TextInput(placeholder='Escribe tu mensaje a...)
    [1] Row
        [0] Button(name='Enviar')
        [1] Button(name='Terminar Chat')
        [2] LoadingSpinner(height=30, width=30)
    [2] Column(sizing_mode='stretch_width')